### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [11]:
#find the number of unique schools in the school name column
total_schools = school_data_complete['school_name'].nunique()

#find the number students we have in total
total_students = school_data_complete['student_name'].count()

#find buget of all schools put together
Total_Budget = school_data['budget'].sum()

#average math score 
Math_average = school_data_complete.math_score.mean()

#average reading score
Reading_average = school_data_complete.reading_score.mean()

#calculate percent passing math
math_pass = sum(school_data_complete.math_score >= 70)
math_percent = (math_pass/total_students)*100

#calculate percent passing reading
reading_pass = sum(school_data_complete.reading_score >= 70)
reading_percent = (reading_pass/total_students)*100

#calculate overall passing
Overall_passing = sum((school_data_complete.reading_score >= 70) & (school_data_complete.math_score >= 70))
Overall_percent = (Overall_passing/total_students)*100

#combine and display the data we have found
Total_df = {'Total Schools':[total_schools], 'Total Students':[total_students], 'Total Budget':[Total_Budget], 
        'Average Math Score':[Math_average], 'Average Reading Score':[Reading_average], '% Passing Math':[math_percent],
           '% Passing Reading':[reading_percent], '% Overall Passing':[Overall_percent]}
pd.DataFrame(data = Total_df, index = [0])


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [40]:
#gets students information and drops all but the rows we are intested in into a new dataframe
summary_student = student_data[['school_name', 'reading_score', 'math_score']]

#finds total number of students in each school but have to rename the column information is put into and drop another  
School_Total = summary_student['school_name'].value_counts()
School_Total_Students = pd.DataFrame({"Total Students": School_Total})
School_Total_Students = School_Total_Students.rename_axis('school_name').reset_index()

#combines the scores for reading and math by school and adds them all together
sum_school = summary_student.groupby('school_name').sum()

#combines school_data with the sum_school as we need some information from the school_data like budget and school type
school_summary = pd.merge(sum_school, school_data, how="outer", on=["school_name", "school_name"])

#combines school_summary with School_Total_Students giving us one table we can use to find the other bits of data we need
school_summary_complete = pd.merge(school_summary, School_Total_Students, how="outer", on="school_name")

#calculates and adds the Bugdet Per Student to are school_summary_complete dataframe
school_summary_complete['Bugdet Per Student'] = school_summary_complete['budget']/school_summary_complete['Total Students']

#calculates and adds the Average Math Score to are school_summary_complete dataframe
school_summary_complete['Average Math Score'] = school_summary_complete['math_score']/school_summary_complete['Total Students']

#calculates and adds the Average Reading Score to are school_summary_complete dataframe
school_summary_complete['Average Reading Score'] = school_summary_complete['reading_score']/school_summary_complete['Total Students']

#find the number of people who got 70 or over on math and how many students got that amount 
math_above_70 = summary_student[summary_student.math_score >= 70]
sum_school2 = math_above_70.groupby('school_name').count()
sum_school2 = sum_school2.drop(['reading_score'], axis=1)
sum_school2 = sum_school2.rename(columns={'math_score': 'Passed Math'})

#find the number of people who got 70 or over on reading and how many students got that amount 
reading_above_70 = summary_student[summary_student.reading_score >= 70]
sum_school3 = reading_above_70.groupby('school_name').count()
sum_school3 = sum_school3.drop(['math_score'], axis=1)
sum_school3 = sum_school3.rename(columns={'reading_score': 'Passed Reading'})

#find the number of people who got 70 or over on both reading and math and how many students got that amount 
both_above_70 = summary_student.loc[(summary_student['math_score'] >= 70) & (summary_student['reading_score'] >= 70)]
sum_school4 = both_above_70.groupby('school_name').count()
sum_school4 = sum_school4.drop(['math_score'], axis=1)
sum_school4 = sum_school4.rename(columns={'reading_score': 'Overall Pass'})

#get summary data and drop the unwanted columns
school_summary_complete1 = school_summary_complete.drop(['reading_score', 'math_score'], axis=1)

#merge the data frames we with the data on number of students passing reading, writing or overall
school_summary_complete_p = pd.merge(sum_school2, sum_school3, how="outer", on="school_name")
school_summary_complete_p2 = pd.merge(sum_school4, school_summary_complete_p, how="outer", on="school_name")

#merge data about how many students passed reading, writing or overall with the summary data we removed columns from 
school_summary_complete_p3 = pd.merge(school_summary_complete1, school_summary_complete_p2, how="outer", on="school_name")

#calculate the columns we need and add them to our data frame
school_summary_complete_p3['% Passing Math'] = (school_summary_complete_p3['Passed Math']/school_summary_complete_p3['Total Students'])*100
school_summary_complete_p3['% Passing Reading'] = (school_summary_complete_p3['Passed Reading']/school_summary_complete_p3['Total Students'])*100
school_summary_complete_p3['% Passing Overall'] = (school_summary_complete_p3['Overall Pass']/school_summary_complete_p3['Total Students'])*100

#create new data frame that will only contain the data that we need
school_summary_Final = school_summary_complete_p3.drop(['Overall Pass', 'Passed Reading', 'Passed Math', 'School ID', 'size'], axis=1)
school_summary_Final


,school_name,type,budget,Total Students,Bugdet Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,District,3124928,4976,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,1763916,2739,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,3022020,4635,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,248087,427,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,1910635,2917,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,3094650,4761,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
#sort the dataframe we made in school sumarry and have it sorted by decending by % Passing Overall as you go down the list
school_summary_top = school_summary_Final.sort_values("% Passing Overall", ascending=False)

#use head function to show five results of the sorted data and they will be the top five
school_summary_top.head(5)

,school_name,type,budget,Total Students,Bugdet Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
1,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1043130,1635,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,Charter,1319574,2283,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
#sort the dataframe we made in school sumarry and have it sorted by acending by % Passing Overall as you go down the list
school_summary_bottom = school_summary_Final.sort_values("% Passing Overall", ascending=True)

#use head function to show five results of the sorted data and they will be the worst five 
school_summary_bottom.head(5)

,school_name,type,budget,Total Students,Bugdet Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
10,Rodriguez High School,District,2547363,3999,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
2,Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
7,Huang High School,District,1910635,2917,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
5,Hernandez High School,District,3022020,4635,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
8,Johnson High School,District,3094650,4761,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [22]:
#find and separte the data by grade 
grade_9 = student_data[student_data.grade == '9th']
grade_10 = student_data[student_data.grade == '10th']
grade_11 = student_data[student_data.grade == '11th']
grade_12 = student_data[student_data.grade == '12th']

#now group by school and you will have total for of all the scores the students got for each grade by school
grade_9_grouped = grade_9.groupby('school_name').sum()
grade_10_grouped = grade_10.groupby('school_name').sum()
grade_11_grouped = grade_11.groupby('school_name').sum()
grade_12_grouped = grade_12.groupby('school_name').sum()

#drop the unneeded student id from are data
grade_9_grouped = grade_9_grouped.drop(['Student ID'], axis=1)
grade_10_grouped = grade_10_grouped.drop(['Student ID'], axis=1)
grade_11_grouped = grade_11_grouped.drop(['Student ID'], axis=1)
grade_12_grouped = grade_12_grouped.drop(['Student ID'], axis=1)

#rename the columns for each group so its clear that this is a total of the scores for each school
grade_9_grouped = grade_9_grouped.rename(columns={'reading_score': '9th Reading Total', 'math_score': '9th Math Total'})
grade_10_grouped = grade_10_grouped.rename(columns={'reading_score': '10th Reading Total', 'math_score': '10th Math Total'})
grade_11_grouped = grade_11_grouped.rename(columns={'reading_score': '11th Reading Total', 'math_score': '11th Math Total'})
grade_12_grouped = grade_12_grouped.rename(columns={'reading_score': '12th Reading Total', 'math_score': '12th Math Total'})

#merge all the data into one table which has all grades on it
Grades_Grouped_9_10 = pd.merge(grade_9_grouped, grade_10_grouped, how="outer", on=["school_name"])
Grades_Grouped_9_11 = pd.merge(Grades_Grouped_9_10, grade_11_grouped, how="outer", on=["school_name"])
Grades_Grouped_9_12 = pd.merge(Grades_Grouped_9_11, grade_12_grouped, how="outer", on=["school_name"])

#drop information we don't need as in this section we only care about reading score
Grades_Grouped_Reading = Grades_Grouped_9_12[[ '9th Reading Total', '10th Reading Total', '11th Reading Total',
                                        '12th Reading Total']]

#find out the number of students in each school by grade. start
student_count_9th = grade_9.groupby('school_name').count()
student_count_9th = student_count_9th.rename(columns={'Student ID': '9th Grade Students'})
student_count_9th = student_count_9th[[ '9th Grade Students']]

student_count_10th = grade_10.groupby('school_name').count()
student_count_10th = student_count_10th.rename(columns={'Student ID': '10th Grade Students'})
student_count_10th = student_count_10th[[ '10th Grade Students']]

student_count_11th = grade_11.groupby('school_name').count()
student_count_11th = student_count_11th.rename(columns={'Student ID': '11th Grade Students'})
student_count_11th = student_count_11th[[ '11th Grade Students']]

student_count_12th = grade_12.groupby('school_name').count()
student_count_12th = student_count_12th.rename(columns={'Student ID': '12th Grade Students'})
student_count_12th = student_count_12th[[ '12th Grade Students']]
#at the end of this part of code we now have the number of students in each school by grade. end

#now we combine all this infor about students in each grade by school into one frame
student_count_9_10 = pd.merge(student_count_9th, student_count_10th, how="outer", on=["school_name"])
student_count_9_11 = pd.merge(student_count_11th, student_count_9_10, how="outer", on=["school_name"])
student_count_9_12 = pd.merge(student_count_12th, student_count_9_11, how="outer", on=["school_name"])

#now merge that with dataframe about total students score by grade with number of students per grade
Grades_Reading_Final = pd.merge(student_count_9_12, Grades_Grouped_Reading, how="outer", on=["school_name"])

#using the frame we just made we can make a new colums and calculate the data that we want
Grades_Reading_Final['9th Grade Average'] = Grades_Reading_Final['9th Reading Total']/Grades_Reading_Final['9th Grade Students']
Grades_Reading_Final['10th Grade Average'] = Grades_Reading_Final['10th Reading Total']/Grades_Reading_Final['10th Grade Students']
Grades_Reading_Final['11th Grade Average'] = Grades_Reading_Final['11th Reading Total']/Grades_Reading_Final['11th Grade Students']
Grades_Reading_Final['12th Grade Average'] = Grades_Reading_Final['12th Reading Total']/Grades_Reading_Final['12th Grade Students']

#Create new data frame which just displays the data we want to show
Grades_Reading_Finished = Grades_Reading_Final[['9th Grade Average', '10th Grade Average', '11th Grade Average',
                                        '12th Grade Average']]
Grades_Reading_Finished


,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [23]:
#using a frame from previous section we can make a dataframe with the math information only instead of reading info
Grades_Grouped_Math = Grades_Grouped_9_12[['9th Math Total', '10th Math Total', '11th Math Total', '12th Math Total']]

#we don't need to calculate total students again as we did that in previous section so can just combine it with our math frame
Grades_Math_Final = pd.merge(Grades_Grouped_Math, student_count_9_12, how="outer", on=["school_name"])

#calculate and add new columns with the info we want to the frame we made above 
Grades_Math_Final['9th Grade Average'] = Grades_Math_Final['9th Math Total']/Grades_Math_Final['9th Grade Students']
Grades_Math_Final['10th Grade Average'] = Grades_Math_Final['10th Math Total']/Grades_Math_Final['10th Grade Students']
Grades_Math_Final['11th Grade Average'] = Grades_Math_Final['11th Math Total']/Grades_Math_Final['11th Grade Students']
Grades_Math_Final['12th Grade Average'] = Grades_Math_Final['12th Math Total']/Grades_Math_Final['12th Grade Students']

#now we have all the data we can get ride of new unwanted colomns and by setting up a new frame
Grades_Math_Finished = Grades_Math_Final[['9th Grade Average', '10th Grade Average', '11th Grade Average',
                                          '12th Grade Average']]
Grades_Math_Finished


,9th Grade Average,10th Grade Average,11th Grade Average,12th Grade Average
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [79]:
school_summary_Final['Spending Ranges (Per Student)'] = pd.cut(school_summary_Final['Bugdet Per Student'], 
                                                               [0, 585, 630, 645, 680])


School_ps_Total = school_summary_Final['Spending Ranges (Per Student)'].value_counts()
School_ps_Total_Fin = pd.DataFrame({"Number": School_ps_Total})
School_ps_Total_Fin = School_ps_Total_Fin.rename_axis('Spending Ranges (Per Student)').reset_index()

Sum_ps = school_summary_Final.groupby('Spending Ranges (Per Student)').sum()
Sum_ps = Sum_ps.rename_axis('Spending Ranges (Per Student)').reset_index()

School_ps = pd.merge(Sum_ps, School_ps_Total_Fin, how="outer", on=["Spending Ranges (Per Student)"])

School_ps['Math Score Average'] = School_ps['Average Math Score']/School_ps['Number']
School_ps['Reading Average Score'] = School_ps['Average Reading Score']/School_ps['Number']
School_ps['Math % Passing'] = School_ps['% Passing Math']/School_ps['Number']
School_ps['Reading % Passing'] = School_ps['% Passing Reading']/School_ps['Number']
School_ps['Overall % Passing'] = School_ps['% Passing Overall']/School_ps['Number']

School_ps_end = School_ps[['Spending Ranges (Per Student)', 'Math Score Average', 'Reading Average Score', 'Math % Passing',
                          'Reading % Passing', 'Overall % Passing']]
School_ps_end


,Spending Ranges (Per Student),Math Score Average,Reading Average Score,Math % Passing,Reading % Passing,Overall % Passing
0,"(0, 585]",83.455399,83.933814,93.460096,96.610877,90.369459
1,"(585, 630]",81.899826,83.155286,87.133538,92.718205,81.418596
2,"(630, 645]",78.518855,81.624473,73.484209,84.391793,62.857656
3,"(645, 680]",76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [92]:
school_summary_complete['school size'] = pd.cut(school_summary_complete['size'], [0, 1000, 2000, 5000])

School_size_Total = school_summary_complete['school size'].value_counts()
School_size_Total_Fin = pd.DataFrame({"Number": School_Size_Total})
School_size_Total_Fin = School_size_Total_Fin.rename_axis('school size').reset_index()

Sum_size = school_summary_complete.groupby('school size').sum()
Sum_size = Sum_size.rename_axis('school size').reset_index()

School_size = pd.merge(Sum_size, School_size_Total_Fin, how="outer", on=["school size"])
School_size


,school size,reading_score,math_score,School ID,size,budget,Total Students,Bugdet Per Student,Average Math Score,Average Reading Score,Number
0,"(0.0, 1000.0]",116640.0,116438.0,17.0,1389.0,833945.0,1389.0,1190.0,167.643196,167.859687,NaN
1,"(1000.0, 2000.0]",714723.0,710502.0,36.0,8522.0,5147986.0,8522.0,3028.0,416.873419,419.322192,NaN
2,"(2000.0, 5000.0]",2375792.0,2266917.0,52.0,29259.0,18667497.0,29259.0,5083.0,621.971332,650.755942,NaN
3,"(0.0, 585.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,"(585.0, 630.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
5,"(630.0, 645.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
6,"(645.0, 680.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


## Scores by School Type

* Perform the same operations as above, based on school type

In [33]:
#group all the shools by type from a pevious data frame that has grade info we need and then reduce columns to needed ones
School_Type_Sum = school_summary_Final.groupby('type').sum()
School_Type_Summary = School_Type_Sum[[ 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                       '% Passing Overall']]

#find out how many of each type of school there is and turn result into a dataframe
School_Total_type = school_data['type'].value_counts()
School_Total_type_df = pd.DataFrame({"Total Type": School_Total_type})
School_Total_type_df = School_Total_type_df.rename_axis('type').reset_index()

#merge the two dataframes into one where we can manipulate and fine the information needed
School_Type_Final = pd.merge(School_Total_type_df, School_Type_Summary, how="outer", on=["type"])

#calculate and add the columns to are frame data 
School_Type_Final['Overall Passing %'] = School_Type_Final['% Passing Overall']/School_Type_Final['Total Type']
School_Type_Final['Reading Passing %'] = School_Type_Final['% Passing Reading']/School_Type_Final['Total Type']
School_Type_Final['Math Passing %'] = School_Type_Final['% Passing Math']/School_Type_Final['Total Type']
School_Type_Final['Math Average Score'] = School_Type_Final['Average Math Score']/School_Type_Final['Total Type']
School_Type_Final['Reading Average Score'] = School_Type_Final['Average Reading Score']/School_Type_Final['Total Type']

#create and print a new dataframe for our final result with only the columns we need
School_Type_End = School_Type_Final[['type', 'Math Average Score', 'Reading Average Score', 'Math Passing %',
                                     'Reading Passing %', 'Overall Passing %']]
School_Type_End


,type,Math Average Score,Reading Average Score,Math Passing %,Reading Passing %,Overall Passing %
0,Charter,83.473852,83.896421,93.620830,96.586489,90.432244
1,District,76.956733,80.966636,66.548453,80.799062,53.672208
